# Azure AI Video Retrieval APIs

To use the Video Retrieval APIs in a typical pattern, you'll follow these steps:

1. Create an index using PUT - Create an index.
2. Add video documents to the index using PUT - CreateIngestion.
3. Wait for the ingestion to complete, checking with GET - ListIngestions.
4. Search for a keyword or phrase using POST - SearchByText.

## Load Azure Configuration

In [ ]:
import os
azure_computer_vision_endpoint = os.environ["AZURE_COMPUTER_VISION_ENDPOINT"]
azure_computer_vision_key = os.environ["AZURE_COMPUTER_VISION_KEY"]
VIDEO_INDEX = "my-video-index-01"

## Step 1 Create Index

In [ ]:
import requests
import json

# Define the endpoint URL and subscription key
index_name = VIDEO_INDEX
endpoint_url = f"{azure_computer_vision_endpoint}/computervision/retrieval/indexes/{index_name}?api-version=2023-05-01-preview"
subscription_key = azure_computer_vision_key

# Define the headers
headers = {
    "Ocp-Apim-Subscription-Key": subscription_key,
    "Content-Type": "application/json"
}

# Define the payload
payload = {
    "metadataSchema": {
        # Define the fields that you want to include in the index
        "fields": [
            {
                "name": "cameraId",
                "searchable": False,
                "filterable": True,
                "type": "string"
            },
            {
                "name": "timestamp",
                "searchable": False,
                "filterable": True,
                "type": "datetime"
            }
        ]
    },
    "features": [
        # Define the features that you want to include in the index
        # https://learn.microsoft.com/en-us/azure/ai-services/computer-vision/reference-video-search#featuremodel
        {
            "name": "vision",
            "domain": "surveillance"
        },
        {
            "name": "speech"
        }
    ]
}

# Make the PUT request
response = requests.put(endpoint_url, headers=headers, json=payload)

# Print the response
print(response.status_code)
# Pretty-print the response JSON if the request was successful
print(json.dumps(response.json(), indent=4))

## Step 2: Add video files to the index

In [ ]:
import requests
import json

# Define the endpoint URL and subscription key
index_name = VIDEO_INDEX
endpoint_url = f"{azure_computer_vision_endpoint}/computervision/retrieval/indexes/{index_name}/ingestions/my-ingestion?api-version=2023-05-01-preview"
subscription_key = azure_computer_vision_key

headers = {
    "Ocp-Apim-Subscription-Key": subscription_key,
    "Content-Type": "application/json"
}

payload = {
    # Define the videos that you want to ingest
    "videos": [
    {
      "mode": "add",
      "documentId": "001",
      "documentUrl": "https://linkedinstorage01.blob.core.windows.net/videos/mercedes.mp4",
      "metadata": {
        "cameraId": "camera1",
        "timestamp": "2025-03-07 16:40:33"
      }
    },
    {
      "mode": "add",
      "documentId": "002",
      "documentUrl": "https://ziggystorage01.blob.core.windows.net/videos/dow.mp4",
      "metadata": {
        "cameraId": "camera2",
        "timestamp": "2025-03-07 17:25:12"
      }
    }
  ]
}

response = requests.put(endpoint_url, headers=headers, json=payload)
print(response.status_code)
# Pretty-print the response JSON if the request was successful
print(json.dumps(response.json(), indent=4))

## Step 3: Wait for ingestion to complete

Wait for the status to be completed

In [ ]:
import requests
import json

# Define the endpoint URL and subscription key
index_name = VIDEO_INDEX
endpoint_url = f"{azure_computer_vision_endpoint}/computervision/retrieval/indexes/{index_name}/ingestions?api-version=2023-05-01-preview&$top=20"
subscription_key = azure_computer_vision_key

headers = {
    "Ocp-Apim-Subscription-Key": subscription_key
}

# Make the GET request
response = requests.get(endpoint_url, headers=headers)

# Check the response status code
if response.status_code == 200:
    # Pretty-print the response JSON if the request was successful
    print(json.dumps(response.json(), indent=4))
else:
    # Print the error message if the request failed
    print(f"Error: {response.status_code}")
    print(response.text)

## [Optional] List Documents

As an optional step, we can list the documents ingested

In [ ]:
import requests
import json

# Define the endpoint URL and subscription key
index_name = VIDEO_INDEX
endpoint_url = f"{azure_computer_vision_endpoint}/computervision/retrieval/indexes/{index_name}/documents?api-version=2023-05-01-preview"
subscription_key = azure_computer_vision_key

headers = {
    "Ocp-Apim-Subscription-Key": subscription_key
}

# Make the GET request
response = requests.get(endpoint_url, headers=headers)

# Check the response status code
if response.status_code == 200:
    # Pretty-print the response JSON if the request was successful
    print(json.dumps(response.json(), indent=4))
else:
    # Print the error message if the request failed
    print(f"Error: {response.status_code}")
    print(response.text)

## Step 4: Perform searches with metadata - Vision Feature

In [ ]:
import requests
import json

# Define the endpoint URL and subscription key
index_name = VIDEO_INDEX
endpoint_url = f"{azure_computer_vision_endpoint}/computervision/retrieval/indexes/{index_name}:queryByText?api-version=2023-05-01-preview"
subscription_key = azure_computer_vision_key

headers = {
    "Ocp-Apim-Subscription-Key": subscription_key,
    "Content-Type": "application/json"
}

# Define the payload
payload = {
  "queryText": "mercedes benz",
  "filters": {
    "stringFilters": [
      {
        "fieldName": "cameraId",
        "values": ["camera1","camera2"]
      }
    ],
    "featureFilters": ["vision"]
  }
}

# Make the POST request
response = requests.post(endpoint_url, headers=headers, json=payload)

# Check the response status code
if response.status_code == 200:
    # Pretty-print the response JSON if the request was successful
    print(json.dumps(response.json(), indent=4))
else:
    # Print the error message if the request failed
    print(f"Error: {response.status_code}")
    print(response.text)

## Step 4: Perform searches with metadata - Speech Feature

In [ ]:
import requests
import json

# Define the endpoint URL and subscription key
index_name = VIDEO_INDEX
endpoint_url = f"{azure_computer_vision_endpoint}/computervision/retrieval/indexes/{index_name}:queryByText?api-version=2023-05-01-preview"
subscription_key = azure_computer_vision_key

headers = {
    "Ocp-Apim-Subscription-Key": subscription_key,
    "Content-Type": "application/json"
}

# Define the payload
payload = {
  "queryText": "leak",
  "dedup": False,
  "filters": {
    "stringFilters": [
      {
        "fieldName": "cameraId",
        "values": ["camera1","camera2"]
      }
    ],
    "featureFilters": ["speech"]
  }
}

# Make the POST request
response = requests.post(endpoint_url, headers=headers, json=payload)

# Check the response status code
if response.status_code == 200:
    # Pretty-print the response JSON if the request was successful
    print(json.dumps(response.json(), indent=4))
else:
    # Print the error message if the request failed
    print(f"Error: {response.status_code}")
    print(response.text)

## Step 5: Delete the Index

In [ ]:
import requests

# Define the endpoint URL and subscription key
index_name = VIDEO_INDEX
endpoint_url = f"{azure_computer_vision_endpoint}/computervision/retrieval/indexes/{index_name}?api-version=2023-05-01-preview"
subscription_key = azure_computer_vision_key

headers = {
    "Ocp-Apim-Subscription-Key": subscription_key
}

# Make the DELETE request
response = requests.delete(endpoint_url, headers=headers)

# Check the response status code
if response.status_code == 204:
    print("Index deleted successfully.")
else:
    print(f"Error: {response.status_code}")
    print(response.text)